In [1]:
import py_stringmatching as sm 
import string
import nltk
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np 
from sklearn.metrics import precision_score, recall_score, classification_report
from collections import Counter
from tqdm import tqdm
import itertools
import multiprocess as mp
import collections

## Question 1
### Problem Statement:3
Identify matches for "United States Navy", "IBM", and "Univ. of Calif." in the "Patentholders.csv" file.

### Process Overview

First, I load in the "Patentholders.csv" data set and begin some basic text pre-processing. I remove punctuation and convert all strings to lowercase. Then, I use a whitespace tokenizer and a 2-gram tokenizer to use for string similarity measures. I then remove duplicates to reduce processing time (though it is not substantial).

Then, I apply the same preprocessing steps to the strings I am trying to match against. 

Following the text pre-processing, I used some prior knowledge and heuristics to identify "ground truth" matches for each of the strings enumerated above so I can evaluate the performance of my string matching protocol. The ground truth may not be perfect, but it definitely helps evaluate which string distance measure performs the best.

Finally, I test four different string distance measures. These includes Smith Waterman, Jaccard, Overlap Coefficient, and Jaro Winkler. After comparing the results of different measures, I concluded that the Overlap Coefficient using 2-grams returned the best matches for all three strings I was attempting to identify

In [2]:
data = pd.read_csv('Patentholders.csv', header=None, names=["Name"])

### Text pre-processing
1. Removing punctuation
1. Transform all strings to lower case

In [3]:
# Punctuation Remover function
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

# create a qgram tokenizer
q=2
qg3_tok = sm.QgramTokenizer(qval=q)

# create a whitespace tokenizer
ws_tok = sm.WhitespaceTokenizer()

In [4]:
strings = ["United States Navy",
           "IBM",
           "Univ. of Calif."
          ]
strings_df = pd.DataFrame(strings)
strings_df.columns = ['Name']
strings_df

strings_df['Name'] = strings_df['Name'].str.replace('IBM', 'International Business Machines')

strings_df["Clean_Name"] = strings_df['Name'].str.lower()
strings_df["Clean_Name"] = strings_df["Clean_Name"].apply(lambda x:remove_punctuation(x))
strings_df["Word_Tokens"] = strings_df["Clean_Name"].map(ws_tok.tokenize)
strings_df["processed_ngrams"] = strings_df["Clean_Name"].map(qg3_tok.tokenize)

strings_df

,Name,Clean_Name,Word_Tokens,processed_ngrams
0,United States Navy,united states navy,"[united, states, navy]","[#u, un, ni, it, te, ed, d , s, st, ta, at, t..."
1,International Business Machines,international business machines,"[international, business, machines]","[#i, in, nt, te, er, rn, na, at, ti, io, on, n..."
2,Univ. of Calif.,univ of calif,"[univ, of, calif]","[#u, un, ni, iv, v , o, of, f , c, ca, al, l..."


In [5]:
data['Name'] = data['Name'].str.replace('IBM', 'International Business Machines')

data["Clean_Name"] = data['Name'].str.lower()
data["Clean_Name"] = data["Clean_Name"].apply(lambda x:remove_punctuation(x))
data["Word_Tokens"] = data["Clean_Name"].map(ws_tok.tokenize)
data["processed_ngrams"] = data["Clean_Name"].map(qg3_tok.tokenize)

data = data[['Name','Clean_Name','Word_Tokens','processed_ngrams']]
print("Shape of data before removing duplicates: " + str(data.shape))
data.drop_duplicates("Clean_Name",inplace=True)
print("Shape of data after removing duplicates: " + str(data.shape))

Shape of data before removing duplicates: (7620, 4)
Shape of data after removing duplicates: (7559, 4)


In [6]:
data.head(3)

,Name,Clean_Name,Word_Tokens,processed_ngrams
0,Signify Holding BV,signify holding bv,"[signify, holding, bv]","[#s, si, ig, gn, ni, if, fy, y , h, ho, ol, l..."
1,ThyssenKrupp Airport Systems SA,thyssenkrupp airport systems sa,"[thyssenkrupp, airport, systems, sa]","[#t, th, hy, ys, ss, se, en, nk, kr, ru, up, p..."
2,Eurenco,eurenco,[eurenco],"[#e, eu, ur, re, en, nc, co, o$]"


Now, we will apply the same text pre-processing to the strings that we are looking to match

### Creating a "ground truth" or "golden" set of records that *should* match 

In [7]:
data['IBM Target']=np.where(data.Clean_Name.str.contains('ibm|international business machine')==True,1,0)
data['United States Navy Target'] = np.where(data.Clean_Name.str.contains('navy')==True,1,0)
data['Univ. of Calif. Target'] = np.where(((data.Clean_Name.str.contains('california a corp of university of regents'))|
                           (data.Clean_Name.str.contains('university of cal'))
                          )==True,1,0)
data.head(3)

,Name,Clean_Name,Word_Tokens,processed_ngrams,IBM Target,United States Navy Target,Univ. of Calif. Target
0,Signify Holding BV,signify holding bv,"[signify, holding, bv]","[#s, si, ig, gn, ni, if, fy, y , h, ho, ol, l...",0,0,0
1,ThyssenKrupp Airport Systems SA,thyssenkrupp airport systems sa,"[thyssenkrupp, airport, systems, sa]","[#t, th, hy, ys, ss, se, en, nk, kr, ru, up, p...",0,0,0
2,Eurenco,eurenco,[eurenco],"[#e, eu, ur, re, en, nc, co, o$]",0,0,0


### String Distance Measures

I am going to attempt to use a few different string distances metrics and then compare the recall and precision for each. This will help me determine which distance measure is best.

I will attempt to use:
1. Smith Waterman
2. Jaccard
3. OverlapCoefficient
4. Jaro Winkler

In [8]:
sim = sm.SmithWaterman()
SW_data = data.copy()
i=0
for s in strings_df.Clean_Name:
    similarity = (data
                     .apply(lambda x: 
                            sim.get_raw_score(x['Clean_Name'], s),
                            axis=1)
                    )

    col_name =strings_df['Name'][i]
    SW_data[col_name] = similarity
    i=i+1
    

In [9]:
SW_data['IBM pred'] = np.where(SW_data['International Business Machines']==max(SW_data['International Business Machines']),1, 0)
SW_data['United States Navy pred'] = np.where(SW_data['United States Navy']==max(SW_data['United States Navy']),1, 0)
SW_data['Univ. of Calif. pred'] = np.where(SW_data['Univ. of Calif.']==max(SW_data['Univ. of Calif.']),1, 0)

print("IBM Classification Report:")
print(classification_report(SW_data['IBM Target'],SW_data['IBM pred']))
print("United States Navy Classification Report:")
print(classification_report(SW_data['United States Navy Target'],SW_data['United States Navy pred']))
print("Univ. of Calif. Classification Report:")
print(classification_report(SW_data['Univ. of Calif. Target'],SW_data['Univ. of Calif. pred']))

IBM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7557
           1       1.00      1.00      1.00         2

    accuracy                           1.00      7559
   macro avg       1.00      1.00      1.00      7559
weighted avg       1.00      1.00      1.00      7559

United States Navy Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7191
           1       1.00      0.08      0.16       368

    accuracy                           0.96      7559
   macro avg       0.98      0.54      0.57      7559
weighted avg       0.96      0.96      0.94      7559

Univ. of Calif. Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7556
           1       0.33      0.67      0.44         3

    accuracy                           1.00      7559
   macro avg       0

In [10]:
sim = sm.Jaccard()
Jac_data = data.copy()

i=0
for s in strings_df.processed_ngrams:
    similarity = (data
                     .apply(lambda x: 
                            sim.get_raw_score(x['processed_ngrams'], s),
                            axis=1)
                    )

    col_name =strings_df['Name'][i]
    Jac_data[col_name] = similarity
    i=i+1

In [11]:
Jac_data['IBM pred'] = np.where(Jac_data['International Business Machines']==max(Jac_data['International Business Machines']),1, 0)
Jac_data['United States Navy pred'] = np.where(Jac_data['United States Navy']==max(Jac_data['United States Navy']),1, 0)
Jac_data['Univ. of Calif. pred'] = np.where(Jac_data['Univ. of Calif.']==max(Jac_data['Univ. of Calif.']),1, 0)

print("IBM Classification Report:")
print(classification_report(Jac_data['IBM Target'],Jac_data['IBM pred']))
print("United States Navy Classification Report:")
print(classification_report(Jac_data['United States Navy Target'],Jac_data['United States Navy pred']))
print("Univ. of Calif. Classification Report:")
print(classification_report(Jac_data['Univ. of Calif. Target'],Jac_data['Univ. of Calif. pred']))

IBM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7557
           1       1.00      0.50      0.67         2

    accuracy                           1.00      7559
   macro avg       1.00      0.75      0.83      7559
weighted avg       1.00      1.00      1.00      7559

United States Navy Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      7191
           1       1.00      0.00      0.01       368

    accuracy                           0.95      7559
   macro avg       0.98      0.50      0.49      7559
weighted avg       0.95      0.95      0.93      7559

Univ. of Calif. Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7556
           1       1.00      0.33      0.50         3

    accuracy                           1.00      7559
   macro avg       1

In [12]:
sim = sm.OverlapCoefficient()
Overlap_data = data.copy()

i=0
for s in strings_df.processed_ngrams:
    similarity = (data
                     .apply(lambda x: 
                            sim.get_raw_score(x['processed_ngrams'], s),
                            axis=1)
                    )

    col_name =strings_df['Name'][i]
    Overlap_data[col_name] = similarity
    i=i+1

In [13]:
cutoff = 0.73

Overlap_data['IBM pred'] = np.where(Overlap_data['International Business Machines']>cutoff,1, 0)
Overlap_data['United States Navy pred'] = np.where(Overlap_data['United States Navy']>cutoff,1, 0)
Overlap_data['Univ. of Calif. pred'] = np.where(Overlap_data['Univ. of Calif.']>cutoff,1, 0)


print("IBM Classification Report:")
print(classification_report(Overlap_data['IBM Target'],Overlap_data['IBM pred']))
print("United States Navy Classification Report:")
print(classification_report(Overlap_data['United States Navy Target'],Overlap_data['United States Navy pred']))
print("Univ. of Calif. Classification Report:")
print(classification_report(Overlap_data['Univ. of Calif. Target'],Overlap_data['Univ. of Calif. pred']))

IBM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7557
           1       1.00      1.00      1.00         2

    accuracy                           1.00      7559
   macro avg       1.00      1.00      1.00      7559
weighted avg       1.00      1.00      1.00      7559

United States Navy Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7191
           1       0.92      0.84      0.88       368

    accuracy                           0.99      7559
   macro avg       0.96      0.92      0.94      7559
weighted avg       0.99      0.99      0.99      7559

Univ. of Calif. Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7556
           1       0.60      1.00      0.75         3

    accuracy                           1.00      7559
   macro avg       0

In [14]:
sim = sm.JaroWinkler()
JW_data = data.copy()

i=0
for s in strings_df.Clean_Name:
    similarity = (data
                     .apply(lambda x: 
                            sim.get_raw_score(x['Clean_Name'], s),
                            axis=1)
                    )

    col_name =strings_df['Name'][i]
    JW_data[col_name] = similarity
    i=i+1

In [15]:
cutoff = .8

JW_data['IBM pred'] = np.where(JW_data['International Business Machines']>cutoff,1, 0)
JW_data['United States Navy pred'] = np.where(JW_data['United States Navy']>cutoff,1, 0)
JW_data['Univ. of Calif. pred'] = np.where(JW_data['Univ. of Calif.']>cutoff,1, 0)


print("IBM Classification Report:")
print(classification_report(JW_data['IBM Target'],JW_data['IBM pred']))
print("United States Navy Classification Report:")
print(classification_report(JW_data['United States Navy Target'],JW_data['United States Navy pred']))
print("Univ. of Calif. Classification Report:")
print(classification_report(JW_data['Univ. of Calif. Target'],JW_data['Univ. of Calif. pred']))

IBM Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7557
           1       0.06      1.00      0.11         2

    accuracy                           1.00      7559
   macro avg       0.53      1.00      0.55      7559
weighted avg       1.00      1.00      1.00      7559

United States Navy Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      7191
           1       0.67      0.29      0.40       368

    accuracy                           0.96      7559
   macro avg       0.82      0.64      0.69      7559
weighted avg       0.95      0.96      0.95      7559

Univ. of Calif. Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7556
           1       0.12      0.67      0.20         3

    accuracy                           1.00      7559
   macro avg       0

### Conclusion

The best string distance measure I found to identify matches for "IBM","United States Navy" and "Univ. of Calif." was the Overlap Coefficient metric using 2-grams. Based off the ground truth that I established using prior knowledge heuristics for each of the strings, the Overlap Coefficient yielded the best precision, recall, and overall accuracy for all of the strings I was attempting to match. In theory, the best string match algorithm would likely change depending on what string I am attempting to match. I, however, wanted to pick the best general purpose string match method for this question.

+ IBM:
    + The Overlap Coefficient successfully identified "IBM Canada Ltd" as a match, but it missed "International Business Machines Corp".
    + This measure yielded just 100% precision and 100% recall (given that there were only 2 true "IBM" matches)
    + The accuracy was 100%

In [16]:
print(classification_report(Overlap_data['IBM Target'],Overlap_data['IBM pred']))
print("IBM ground truth:")
display(Overlap_data[Overlap_data['IBM Target']==1][['Name','IBM Target','IBM pred']])
print("IBM predicted:")
display(Overlap_data[Overlap_data['IBM pred']==1][['Name','IBM Target','IBM pred']])

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7557
           1       1.00      1.00      1.00         2

    accuracy                           1.00      7559
   macro avg       1.00      1.00      1.00      7559
weighted avg       1.00      1.00      1.00      7559

IBM ground truth:


,Name,IBM Target,IBM pred
3645,International Business Machines Corp,1,1
5184,International Business Machines Canada Ltd,1,1


IBM predicted:


,Name,IBM Target,IBM pred
3645,International Business Machines Corp,1,1
5184,International Business Machines Canada Ltd,1,1


+ United States Navy:
    + The Overlap Coefficient had approximately an 84% recall of true "United States Navy" positives.
    + The Overlap Coefficient had approximately a 92% precision indicating there were 26 false positives based on the ground truth established earlier. However, after looking at many of the "false positives" shown in the table below, it appears that the ground truth established earlier may have missed some true "United States Navy" matches that the Overlap Coefficient method actually matched.
    + The overall accuracy was about 99%


In [17]:
print(classification_report(Overlap_data['United States Navy Target'],Overlap_data['United States Navy pred']))
print("False Positives:")
print(Overlap_data[(Overlap_data['United States Navy Target']==0)&
             (Overlap_data['United States Navy pred']==1)
            ][['Name','United States Navy Target','United States Navy pred']].shape[0])
display(Overlap_data[(Overlap_data['United States Navy Target']==0)&
             (Overlap_data['United States Navy pred']==1)
            ][['Name','United States Navy Target','United States Navy pred']])
print("False Negatives:")
print(Overlap_data[(Overlap_data['United States Navy Target']==1)&
             (Overlap_data['United States Navy pred']==0)
            ][['Name','United States Navy Target','United States Navy pred']].shape[0])
display(Overlap_data[(Overlap_data['United States Navy Target']==1)&
             (Overlap_data['United States Navy pred']==0)
            ][['Name','United States Navy Target','United States Navy pred']])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7191
           1       0.92      0.84      0.88       368

    accuracy                           0.99      7559
   macro avg       0.96      0.92      0.94      7559
weighted avg       0.99      0.99      0.99      7559

False Positives:
26


,Name,United States Navy Target,United States Navy pred
1209,"United States, AS REPRESENTED BY",0,1
1687,UNITED STATES GOVERNMENT AS REPRESENTED BY SEC...,0,1
2127,AERONAUTICS AND SPACE ADMINISTRATION United St...,0,1
2486,UNITED STATES GOVERNMENT IN NAME OF DEFENSE AD...,0,1
2502,NAVAL UNDERSEA WARFARE CENTER DIVISION NEWPORT...,0,1
2596,NAVAL UNDERSEA WARFARE CENTER DIVISION NEWPORT...,0,1
2922,"United States, NAVAL RESEARCH, Secretary of",0,1
3634,"United States, NAVAL UNDERSEA WARFARE CENTER D...",0,1
3784,"United States, NAVT THE, Secretary of",0,1
3864,"UNITED STATES OF AMERICAS NAVT THE, Secretary of",0,1


False Negatives:
60


,Name,United States Navy Target,United States Navy pred
15,NAVY SECRETARY OF UNITED STATE,1,0
38,NAVY SECRETARY OF,1,0
92,"U S NAVY NAVY, Secretary of",1,0
107,NAVY ISLAND PLYWOOD Inc,1,0
191,Department Ofthe Navy,1,0
194,NAVY USA AS REPRESENED BY SECRETARY OF,1,0
482,"NAVY US OF AMERICA THE, Secretary of",1,0
796,US Secretary of Navy,1,0
909,"NAVY US GOVERNMENT, Secretary of",1,0
956,DEPARTMENT OF NAVY - NAWCAD,1,0


+ Univ. of Calif.:
    + The Overlap Coefficient had 100% recall of true "Univ. of Calif." matches.
    + The Overlap Coefficient had approximately a 60% precision indicating there were 2 false positives based on the ground truth established earlier. 
    + The overall accuracy was about 100%

In [18]:
print(classification_report(Overlap_data['Univ. of Calif. Target'],Overlap_data['Univ. of Calif. pred']))
print("Univ. of Calif. ground truth:")
display(Overlap_data[(Overlap_data['Univ. of Calif. Target']==1)
            ][['Name','Univ. of Calif. Target','Univ. of Calif. pred']])
print("Univ. of Calif. predicted:")
display(Overlap_data[(Overlap_data['Univ. of Calif. pred']==1)
            ][['Name','Univ. of Calif. Target','Univ. of Calif. pred']])

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7556
           1       0.60      1.00      0.75         3

    accuracy                           1.00      7559
   macro avg       0.80      1.00      0.87      7559
weighted avg       1.00      1.00      1.00      7559

Univ. of Calif. ground truth:


,Name,Univ. of Calif. Target,Univ. of Calif. pred
144,University of California,1,1
1815,University of California Berkeley,1,1
3021,"CALIFORNIA A CORP OF, University of, Regents of",1,1


Univ. of Calif. predicted:


,Name,Univ. of Calif. Target,Univ. of Calif. pred
144,University of California,1,1
318,Union Oil Co of California,0,1
1815,University of California Berkeley,1,1
2617,University of Southern California USC,0,1
3021,"CALIFORNIA A CORP OF, University of, Regents of",1,1


Below you will find a table with all the predicted matches

In [19]:
final_matches = Overlap_data[(Overlap_data['Univ. of Calif. pred']==1)|
                             (Overlap_data['IBM pred']==1)|
                             (Overlap_data['United States Navy pred']==1)
                            ][['Name','United States Navy pred','IBM pred','Univ. of Calif. pred']]
final_matches

,Name,United States Navy pred,IBM pred,Univ. of Calif. pred
39,"UNTIED STATES OF AMERICA NAVY THE, Secretary of",1,0,0
48,"NAVY GOVERNMENT OR United States, THE, Secreta...",1,0,0
67,SECRETARY OF NAVY AS REPRESENTED BY UNITED STA...,1,0,0
79,"NAVY UNITED STATES THE, Secretary of",1,0,0
87,UNITED STATES DEPARTMENT OF THE NAVY,1,0,0
...,...,...,...,...
7567,"NAVY United States, AS RERESENTED BY SECRETARY OF",1,0,0
7568,"NAVY SECRETARY OF United States, AS REPESENTED",1,0,0
7572,"NAVY United States, SECRETARY AS REPRESENTED BY",1,0,0
7594,SECRETARY OF UNITED STATES NAVY,1,0,0


## Question 2
### Problem Statement:
Using organization names from Patentholders.csv that have the string "Inc" (i.e., incorporated) find similar values in the orgNames.csv data set. Your output should a table where the first column is a name from Patentholders.csv and the second column is a list of matching values from orgNames.csv.

### Loading in `Patentholders.csv` and `orgNames.csv`

In [20]:
patentholders = pd.read_csv('Patentholders.csv', header=None, names=["Name"])
orgNames = pd.read_csv('orgNames.csv', header=None, names=["Name"])

### Text pre-processing of Patentholder.csv
1. Removing punctuation
1. Transform all strings to lower case
1. Tokenizing names with tri-gram and whitespace tokenizers

In [21]:
# Punctuation Remover function
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

# create a qgram tokenizer
q=3
qg3_tok = sm.QgramTokenizer(qval=q)

# create a whitespace tokenizer
ws_tok = sm.WhitespaceTokenizer()

In [22]:
patentholders["Clean_Name"] = patentholders['Name'].str.lower()
patentholders["Clean_Name"] = patentholders["Clean_Name"].apply(lambda x:remove_punctuation(x))
# Removing "Inc" from name

### Filtering to patent holders with "Inc" in their name

Matching only records that contain "Inc" but it needs to follow a whitespace. Otherwise, I find some matches that contain "inc" like "University of Cincinnati" because "inc" is contained within "Cincinnati".

In [23]:
inc_patentholders = patentholders.copy()
inc_patentholders = patentholders[patentholders.Clean_Name.str.contains("\sinc$")]
# inc_patentholders = inc_patentholders.reset_index(drop=True)
# 
#keeping the new index, will be useful later down the line
# inc_patentholders = inc_patentholders.reset_index()
# inc_patentholders = inc_patentholders.rename(columns={'index':'patentholder_id'})
inc_patentholders.shape

(2081, 2)

In [24]:
inc_patentholders["Clean_Name"] = inc_patentholders["Clean_Name"].str.replace("\sinc$",'', regex=True)
inc_patentholders["Word_Tokens"] = inc_patentholders["Clean_Name"].map(ws_tok.tokenize)
inc_patentholders["Clean_Name_no_space"] = inc_patentholders["Clean_Name"].str.replace(" ","")
inc_patentholders["processed_ngrams"] = inc_patentholders["Clean_Name_no_space"].map(qg3_tok.tokenize)
inc_patentholders['n_ngrams'] = inc_patentholders.processed_ngrams.str.len()

inc_patentholders = inc_patentholders[['Name','Clean_Name','Word_Tokens','processed_ngrams','n_ngrams']]
print("Shape of data before removing duplicates: " + str(inc_patentholders.shape))
inc_patentholders.drop_duplicates("Clean_Name",inplace=True)
print("Shape of data after removing duplicates: " + str(inc_patentholders.shape))

inc_patentholders = inc_patentholders.reset_index()
inc_patentholders = inc_patentholders.rename(columns={'index':'patentholder_id'})

inc_patentholders['token_set_patentholder']=inc_patentholders['processed_ngrams'].apply(set)

Shape of data before removing duplicates: (2081, 5)
Shape of data after removing duplicates: (2074, 5)


### Partitioning Patentholder dataframe into sub dataframes based on the number of tokens.

This will allow me to pre-filter the orgNames before attempting to match, thus avoiding a cartesian product between `patentholders` and `orgNames`. For example, the first partition of the `patentholders` contains names that have 7-8 tokens. When I attempt to match with `orgNames`, I can set bounds on the names based on the number of tokens. So, for `patentholders` that have 7-8 tokens, I will compare against orgnames with at least `7-n` tokens and at most `8+n` tokens. 

In [25]:
min_bigrams = inc_patentholders.n_ngrams.min()
max_bigrams = inc_patentholders.n_ngrams.max()
print("Minimum count of bigrams: " + str(min_bigrams))
print("Maximum count of bigrams: " + str(max_bigrams))

# n_batches = max_bigrams-min_bigrams
# rnge = inc_patentholders.n_ngrams.max()-inc_patentholders.n_ngrams.min()

# bigram_increment = rnge//n_batches
# bigram_ranges = [(i,i+bigram_increment-1) for i in range(min_bigrams,max_bigrams,bigram_increment)]
# bigram_ranges[-1] = (bigram_ranges[-1][0],max_bigrams)

partitioned_dataframes = {}
for r in range(min_bigrams,max_bigrams+1):
    range_mask = (inc_patentholders.n_ngrams==r)
#     a = range(r[0],r[1])
#     range_mask = ((inc_patentholders.n_ngrams>=r[0])
#                   & (inc_patentholders.n_ngrams<=r[1]))
    data = inc_patentholders[range_mask]
    partitioned_dataframes[r] = data

Minimum count of bigrams: 4
Maximum count of bigrams: 60


In [26]:
partitioned_dataframes[4]

,patentholder_id,Name,Clean_Name,Word_Tokens,processed_ngrams,n_ngrams,token_set_patentholder
851,3177,HP Inc,hp,[hp],"[##h, #hp, hp$, p$$]",4,"{#hp, p$$, ##h, hp$}"
892,3340,AI Inc,ai,[ai],"[##a, #ai, ai$, i$$]",4,"{i$$, #ai, ##a, ai$}"
1881,6907,CA Inc,ca,[ca],"[##c, #ca, ca$, a$$]",4,"{##c, #ca, a$$, ca$}"


### Text pre-processing of orgNames.csv
1. Removing null records
1. Removing records that are only numbers
1. Removing punctuation
1. Transform all strings to lower case
1. Tokenizing names with tri-gram and whitespace tokenizers

In [27]:
# Removing nulls
orgNames = orgNames[~orgNames.Name.isnull()]

# Removing only numeric entries
orgNames = orgNames[~orgNames.Name.str.contains("^[0-9\s]*$")]

# Removing "Inc" which shouldn't match with anything
orgNames = orgNames[orgNames.Name!='inc']

In [28]:
orgNames["Clean_Name"] = orgNames['Name'].str.lower()
orgNames["Clean_Name"] = orgNames["Clean_Name"].apply(lambda x:remove_punctuation(x))
orgNames["Clean_Name"] = orgNames.Clean_Name.str.replace('\sinc$','',regex=True)
orgNames["Word_Tokens"] = orgNames["Clean_Name"].map(ws_tok.tokenize)
orgNames["Clean_Name_no_space"] = orgNames["Clean_Name"].str.replace(" ","")
orgNames["processed_ngrams"] = orgNames["Clean_Name_no_space"].map(qg3_tok.tokenize)

orgNames = orgNames[['Name','Clean_Name','Word_Tokens','processed_ngrams']]
orgNames.reset_index(inplace=True, drop=True)
orgNames = orgNames.reset_index()
orgNames = orgNames.rename(columns={'index':'orgName_id'})
orgNames['n_ngrams'] = orgNames.processed_ngrams.str.len()
print("Shape of orgNames: " + str(orgNames.shape))
orgNames['token_set_orgName'] = orgNames['processed_ngrams'].apply(set)

Shape of orgNames: (443202, 6)


### Creating a sample golden set to tune string distance

In [30]:
golden_set = orgNames.copy()
golden_set['MACOM Technology Solutions Holdings Inc'] = np.where(golden_set.Name.str.contains('macom tech'),1,0)
golden_set['Maxygen Inc'] = np.where(golden_set.Name.str.contains('maxygen'),1,0)
golden_set['Cavitid Inc'] = np.where(golden_set.Name.str.contains('cavitid'),1,0)
golden_set['Ingalls Shipbuilding Inc'] = np.where(golden_set.Name.str.contains('ingalls ship'),1,0)
golden_set['Oasis Technology Inc'] = np.where(golden_set.Name.str.contains('^oasis tech'),1,0)
golden_set['Aerojet Rocketdyne Inc'] = np.where(golden_set.Name.str.contains('aerojet ro'),1,0)
golden_set['Avanir Pharmaceuticals Inc'] = np.where(golden_set.Name.str.contains('avanir'),1,0)

value_vars = ['orgName_id','Name','MACOM Technology Solutions Holdings Inc',
   'Maxygen Inc','Cavitid Inc','Ingalls Shipbuilding Inc',
   'Oasis Technology Inc','Aerojet Rocketdyne Inc','Avanir Pharmaceuticals Inc']
golden_set = golden_set[value_vars]

golden_set= pd.melt(golden_set, id_vars=['orgName_id','Name'], value_vars=value_vars)
golden_set.columns = ['orgName_id','orgName','patentholder','match']
golden_set['label'] = golden_set['patentholder'].apply(hash)
golden_set = golden_set[golden_set.match==1][['orgName_id','orgName','patentholder','label']]
golden_set

golden_set_patentholders = pd.DataFrame(['MACOM Technology Solutions Holdings Inc',
                            'Maxygen Inc',
                            'Cavitid Inc',
                            'Ingalls Shipbuilding Inc',
                            'Oasis Technology Inc',
                            'Aerojet Rocketdyne Inc',
                            'Avanir Pharmaceuticals Inc'], columns = ['patentholder'])
golden_set = golden_set[(golden_set.orgName != 'aerojet rocketdyne coleman aerospace, inc')&
                        (golden_set.orgName != 'aerojet rocketdyne holdings')
                       ]
golden_set

,orgName_id,orgName,patentholder,label
236340,236340,macom technology solutions,MACOM Technology Solutions Holdings Inc,-5843905523761170206
236341,236341,macom technology solutions holdings inc,MACOM Technology Solutions Holdings Inc,-5843905523761170206
236342,236342,macom technology solutions inc,MACOM Technology Solutions Holdings Inc,-5843905523761170206
687709,244507,maxygen inc,Maxygen Inc,3617926977394716710
956333,69929,cavitid inc,Cavitid Inc,370658653078486758
1522098,192492,ingalls shipbuilding,Ingalls Shipbuilding Inc,5147210623917731294
1522099,192493,ingalls shipbuilding inc,Ingalls Shipbuilding Inc,5147210623917731294
2058564,285756,oasis technology inc,Oasis Technology Inc,4576345790171400863
2058565,285757,"oasis technology, inc",Oasis Technology Inc,4576345790171400863
2223601,7591,aerojet rocketdyne,Aerojet Rocketdyne Inc,4196363101484633534


### Loop over partitioned dataframes and compare each partition to orgNames that satisfy the token size Jaccard restriction AND satisfy the token similarity Jaccard restriction
This imposes a size filter and a token similarity filter to avoid the all-to-all cartesian product

#### Creating a function that returns the string similarity

In [31]:
# Using Jaccard similarity
sim = sm.Jaccard()
cutoff = .7

def get_similar_strings(params):
    i = params[0]
    j = params[1]
    return [i,j,sim.get_sim_score(inc_patentholders[inc_patentholders.patentholder_id==i].processed_ngrams.values[0],orgNames[orgNames.orgName_id==j].processed_ngrams.values[0])]

In [32]:
%%time
# generating multiprocess pool for parallel processing
pool = mp.Pool()


# For each patentholder dataframe partition, compare only orgNames that are within
# +/- p n-grams where p is the Jaccard size restriction based on the cutoff. Put that in the results_dict dictionary
results_dict = {}
for k in tqdm(partitioned_dataframes.keys()):
    # First using the string length cutoff at the cutoff threshold
    mask = ((orgNames.n_ngrams>=k*cutoff) &
            (orgNames.n_ngrams<=k/cutoff))

    orgName_subset = orgNames[mask]
    orgName_ids = orgName_subset.orgName_id
    patentholder_ids = partitioned_dataframes[k].patentholder_id
    paramlist = list(itertools.product(patentholder_ids,orgName_ids))
    if len(paramlist)==0:
        next
    else:
        candidates = pd.DataFrame(paramlist)
        candidates.columns = ['patentholder_id','orgName_id']

        candidates = (candidates
                      .merge(partitioned_dataframes[k],'left','patentholder_id')
                      .merge(orgNames, 'left','orgName_id')
                     )

        # Checking the set of tokens have a required overlap before applying any string distance matching

        candidates['n_common_tokens'] =(
            [len(set(a).intersection(b)) for a, b in zip(candidates.token_set_patentholder, candidates.token_set_orgName)]
        )
        
        # Using Jaccard, coming up with the required number of common token value for each string
        candidates['n_common_tokens_needed'] = (cutoff/(1+cutoff))*(candidates.n_ngrams_x+candidates.n_ngrams_y)

        # Only evaluate tupes of patentholders and orgNames that satisfy the required tokens in common condition
        candidates = candidates[candidates.n_common_tokens>=candidates.n_common_tokens_needed]
        candidate_ids = list(zip(candidates.patentholder_id,candidates.orgName_id))

        # Distribute the parameter sets evenly across the cores
        res  = np.array(pool.map(get_similar_strings,candidate_ids))

        # Only keeping matches that have Jaccard similarity > cutoff value
        if len(res)==0:
            next
        else: 
            res = res[res[:,2]>cutoff]
            results_dict[k] = (res) #res

100%|███████████████████████████████████████████| 57/57 [15:19<00:00, 16.13s/it]

CPU times: user 12min 54s, sys: 2min 3s, total: 14min 57s
Wall time: 15min 19s


### Generating the final output table

In [33]:
similarity_filter = cutoff

result_df = pd.DataFrame(
    np.concatenate(
        list(results_dict.values())[0:]),
    columns = ['patentholder_id','orgName_id','similarity']
)
result_df.patentholder_id = result_df.patentholder_id.astype(int)
result_df.orgName_id = result_df.orgName_id.astype(int)
result_df = result_df[result_df.similarity>similarity_filter]

# Filtering to a similarity minimum and joining on actual names
matches = (result_df
           .merge(
               inc_patentholders[['patentholder_id','Name']].rename(columns={'Name':'patentholder'})
               ,'left'
               ,'patentholder_id')
           .merge(orgNames[['orgName_id','Name']].rename(columns={'Name':'orgName'})
                 ,'left'
                 ,'orgName_id')
          )

matches = matches[['patentholder','orgName','orgName_id']]
matches

# Creating the output in the requested data structure
final_results = pd.DataFrame(matches.groupby('patentholder')['orgName'].apply(list))
final_results = final_results.reset_index() 
final_results.columns = ['patentholder_name','orgName']
final_results

,patentholder_name,orgName
0,2040422 Ontario Inc,[2040422 ontario inc]
1,21CT Inc,"[21ct inc, 21ct, inc]"
2,21st Century Fox America Inc,"[21st century america, 21st century fox americ..."
3,21st Century Systems Inc,"[21st century systems inc, 21st century system..."
4,2236008 Ontario Inc,[2236008 ontario inc]
...,...,...
1685,nDosa Tech Inc,[ndosa tech inc]
1686,nLight Inc,"[nlight, inc]"
1687,nuTonomy Inc,"[nutonomy, nutonomy inc]"
1688,phA Environmental Restoration Inc,[pha environmental restoration inc]


In [34]:
final_match_df = inc_patentholders[['Name']]
final_match_df.columns = ['patentholder_name']
final_match_df = final_match_df.merge(final_results,'left','patentholder_name')
final_match_df.to_csv('Q2_output.csv')
# final_match_df

### Evaluating Precision and Recall on sample golden set

In [35]:
final_match_eval = matches.copy()
final_match_eval = final_match_eval[final_match_eval.patentholder.isin(value_vars)]

final_match_eval['pred_patentholder_name'] = final_match_eval['patentholder']
final_match_eval = final_match_eval[['orgName_id','orgName','pred_patentholder_name']]
final_match_eval = final_match_eval.merge(golden_set,'outer','orgName')
final_match_eval = final_match_eval.fillna('None')
final_match_eval['pred_patentholder_name_label'] = final_match_eval.pred_patentholder_name.apply(hash)
final_match_eval
print("Precision: "+str(precision_score(final_match_eval['label'],final_match_eval['pred_patentholder_name_label'],average='macro')))
print("Recall: "+str(recall_score(final_match_eval['label'],final_match_eval['pred_patentholder_name_label'],average='macro')))

Precision: 1.0
Recall: 1.0


Although the Precision and Recall are 100% on my golden set, this is *just a sample* of patentholder matches. In reality, the actual Precision and Recall are likely less than 100%, but using this golden set I was able to tune my Jaccard threshold to get the best results.

## Question 3
### Problem Statement:
Using the uscities.csv data set find groups of cities that have approximately the same latitude (within 0.5 degrees of each other). Your output will be a 2-column table where the first column is a city name (with lat-long in parentheses) and the second column is a list of cities that fall in the same latitude band also represented as city name (with lat-long in parentheses).

In [48]:
uscities = pd.read_csv('uscities.csv')
uscities_df = uscities[['city','lat','lng']]
uscities_df['city'] = (uscities_df['city']
                          +' ('+uscities_df['lat'].astype(str)
                          +','+uscities_df['lng'].astype(str)+')'
                         )
uscities_df

,city,lat,lng
0,"New York (40.6943,-73.9249)",40.6943,-73.9249
1,"Los Angeles (34.1139,-118.4068)",34.1139,-118.4068
2,"Chicago (41.8373,-87.6862)",41.8373,-87.6862
3,"Miami (25.7839,-80.2102)",25.7839,-80.2102
4,"Dallas (32.7936,-96.7662)",32.7936,-96.7662
...,...,...,...
28333,"Gross (42.9461,-98.5697)",42.9461,-98.5697
28334,"Lotsee (36.1334,-96.2091)",36.1334,-96.2091
28335,"The Ranch (47.3198,-95.6952)",47.3198,-95.6952
28336,"Shamrock (35.9113,-96.5772)",35.9113,-96.5772


In [49]:
c1,c2 = 0.25,0.25

### Phase 1: Building

In [50]:
R = uscities_df.copy()
R['ri'] = R.lat - R.lat%(c1+c2)
R['hi'] = R.ri.apply(hash)

hi_lookup = pd.DataFrame(R.groupby('hi')['city'].apply(list))

#This is my hash-table with many many collisions
hi_lookup

,city
hi,
18,"[San Juan (18.4037,-66.0636), Bayamón (18.3793..."
19,"[Hawaiian Ocean View (19.0959,-155.775), Capta..."
20,"[Waimea (20.0124,-155.6378), Honokaa (20.0746,..."
21,"[Honolulu (21.3294,-157.846), East Honolulu (2..."
22,"[Kapaa (22.091,-159.352), Wailua Homesteads (2..."
...,...
1152921504606847041,"[Buckland (65.9784,-161.1341), Huslia (65.7002..."
1152921504606847042,"[Kotzebue (66.8766,-162.5231), Selawik (66.598..."
1152921504606847043,"[Noatak (67.5987,-163.0309), Kivalina (67.733,..."


### Phase 2: Probing

In [51]:
S = uscities_df.copy()
S['sj1'] = S.lat - S.lat%(c1+c2)
S['sj2'] = np.where(S.lat < S.sj1+c2, S.sj1-(c1+c2), S.sj1+(c1+c2))
S['hj1'] = S.sj1.apply(hash)
S['hj2'] = S.sj2.apply(hash)
S.sort_values('lat').head(10)

S = S[['hj1','hj2','city','lat']]
S


,hj1,hj2,city,lat
0,1152921504606847016,40,"New York (40.6943,-73.9249)",40.6943
1,34,1152921504606847009,"Los Angeles (34.1139,-118.4068)",34.1139
2,1152921504606847017,42,"Chicago (41.8373,-87.6862)",41.8373
3,1152921504606847001,26,"Miami (25.7839,-80.2102)",25.7839
4,1152921504606847008,33,"Dallas (32.7936,-96.7662)",32.7936
...,...,...,...,...
28333,1152921504606847018,43,"Gross (42.9461,-98.5697)",42.9461
28334,36,1152921504606847011,"Lotsee (36.1334,-96.2091)",36.1334
28335,47,1152921504606847023,"The Ranch (47.3198,-95.6952)",47.3198
28336,1152921504606847011,36,"Shamrock (35.9113,-96.5772)",35.9113


### Phase 3: Join

In [53]:
%%time
final = pd.melt(S, id_vars=['city','lat'], value_vars=['hj1','hj2'])
final

CPU times: user 14.5 ms, sys: 4.65 ms, total: 19.1 ms
Wall time: 18 ms


,city,lat,variable,value
0,"New York (40.6943,-73.9249)",40.6943,hj1,1152921504606847016
1,"Los Angeles (34.1139,-118.4068)",34.1139,hj1,34
2,"Chicago (41.8373,-87.6862)",41.8373,hj1,1152921504606847017
3,"Miami (25.7839,-80.2102)",25.7839,hj1,1152921504606847001
4,"Dallas (32.7936,-96.7662)",32.7936,hj1,1152921504606847008
...,...,...,...,...
56671,"Gross (42.9461,-98.5697)",42.9461,hj2,43
56672,"Lotsee (36.1334,-96.2091)",36.1334,hj2,1152921504606847011
56673,"The Ranch (47.3198,-95.6952)",47.3198,hj2,1152921504606847023
56674,"Shamrock (35.9113,-96.5772)",35.9113,hj2,36


In [47]:
final = final[['city','lat','value']]
final.columns = ['city','lat','hj']
final = final.merge(hi_lookup,'left',left_on='hj',right_on='hi')


final = final[['city_x','lat','city_y']]
final = final.explode('city_y')
final = final.merge(uscities_df,'left',left_on='city_y',right_on='city')
final = final[abs(final.lat_x-final.lat_y)<c1]

final = pd.DataFrame(final[['city_x','city_y']].groupby('city_x')['city_y'].apply(list))
final = final.reset_index()
final.columns = ['City','Cities in Same Latitute Band']

final

CPU times: user 17.5 s, sys: 9.53 s, total: 27.1 s
Wall time: 27.6 s


,City,Cities in Same Latitute Band
0,"Aaronsburg (40.9042,-77.4513)","[New York (40.6943,-73.9249), Queens (40.7498,..."
1,"Abanda (33.0926,-85.5253)","[Denton (33.2176,-97.1419), Plano (33.0502,-96..."
2,"Abbeville (29.975,-92.1265)","[Houston (29.7863,-95.3889), Port Arthur (29.8..."
3,"Abbeville (31.5664,-85.2528)","[Waco (31.5598,-97.1881), Albany (31.5776,-84...."
4,"Abbeville (31.9925,-83.3068)","[El Paso (31.8479,-106.4309), Odessa (31.8831,..."
...,...,...
28333,"Zumbrota (44.295,-92.6734)","[Eugene (44.0563,-123.1173), Appleton (44.2779..."
28334,"Zuni Pueblo (35.0708,-108.8484)","[Charlotte (35.208,-80.8304), Memphis (35.1046..."
28335,"Zurich (39.2322,-99.4347)","[Baltimore (39.3051,-76.6144), Cincinnati (39...."
28336,"Zwingle (42.2972,-90.6874)","[Boston (42.3188,-71.0846), Detroit (42.3834,-..."
